<a href="https://colab.research.google.com/github/entanglesoftware/ai-learning/blob/main/langchain/prompting_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain==0.0.288 openai==0.27.8 tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.13
    Uninstalling langsmith-0.3.13:
      Successfully uninstalled langsmith-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Succe

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [5]:
# ============ PROMPT TEMPLATE ==============

from langchain import LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = """ Answer the question based on the context below. If the question can not be answered using
the information provided, answer with "I don't know".

Context: Quantam Computing is an emerging field that leverages quantam mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """


prompt_template = PromptTemplate(input_variables=["query"], template=template)

chain = LLMChain(llm=llm, prompt=prompt_template)

input_data = {"query": "What is the main advantage of quantam computing over classical computing?"}

result = chain.run(input_data)

print("Question:", input_data["query"])
print("Answer:", result)

Question: What is the main advantage of quantam computing over classical computing?
Answer: The main advantage of quantum computing over classical computing is the ability to solve complex problems faster.


In [11]:
# ============ EXAMPLE PROMPT TEMPLATE ==============

from langchain import LLMChain, FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"},
]

example_template = """
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)

dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n",
)

chain = LLMChain(llm=llm, prompt=dynamic_prompt)

input_data = {"input": "tiger"}

result = chain.run(input_data)

print("Animal:", input_data["input"])
print("Habitat:", result)

### @TODO with the local file system in JSON OR YAML format

dynamic_prompt.save("awesome_prompt.json")

Animal: tiger
Habitat: tropical rainforests, grasslands, and mangrove swamps


In [14]:
# ============ EXAMPLE PROMPT TEMPLATE WITH FEW SHOT ==============

from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

examples = [
    {
        "query": "How do I becomoe a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    },
    {
        "query": "why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain.",
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI assistant.
The assistant is typically sarcastic and witty, producing creative and funny responses
to users' questions. Here are some examples:
"""

suffix = """
User: {query}
AI: """


few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

input_data = {"query": "How can I learn quantum computing?"}

result = chain.run(input_data)

print(result)

Step 1: Watch every episode of Rick and Morty. Step 2: Realize you still have no idea what quantum computing is. Step 3: Google it.


In [25]:
# ============ RANDOMIZED EXAMPLE TEMPLATE WITH FEW SHOT ==============

examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    },
    {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    },
    {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    },
    {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    },
    {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    },
    {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    },
    {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    },
    {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump.",
    },
    {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies.",
    },
    {
        "query": "What is the largest mammal in the world?",
        "answer": "Blue Whale, the largest land animal on Earth.",
    },
    {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omellete.",
    },
    {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa.",
    }
]

from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # this sets the max length that examples should be
)

dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)


from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

chain = LLMChain(llm=llm, prompt=dynamic_prompt_template)

input_data = {"query": "Who invented the telephone?"}

result = chain.run(input_data)

print(result)

Alexander Graham Bell, but I like to think it was actually a group of pigeons with tin cans and string.
